In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
from time import time
from scipy.special import erf

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_1deg.npy')
pm_data_noerr = np.load('npy/pm_LAE_1deg.npy')
f_name = 'Source_cats/Source_cat_LAE_1deg.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/errors5Sigma.npy')[:,1]*4

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
nb_c = 26
N_nb = 6

pm_err = (np.ones(pm_data.shape).T * errors).T
cont_est, cont_err = synthetic_BB_estimation(
    pm_data, pm_err, nb_c, N_nb, tcurves['t'], tcurves['w'], w_central
    )
real_cont_value, _ = synthetic_BB_estimation(
    pm_data_noerr, pm_err, nb_c, 2, tcurves['t'], tcurves['w'], w_central, False
    )

In [ ]:
## Plots
for mock_n in np.random.randint(0, pm_data.shape[1], 20):
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots(figsize = (12, 8))
    ax.plot(mock['w_Arr'], mock['SEDs_no_line'][mock_n], c='orange', lw=2, zorder=-1)
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], c='gray', fmt='.')
    ax.scatter(w_central[nb_c], real_cont_value[mock_n], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4],
                xerr=bb_fwhm[-4]/2, yerr=errors[-4],
                fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], yerr=errors[-3],
                xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], yerr=errors[-2],
                xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], yerr=errors[-1],
                xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    ax.errorbar(w_central[nb_c]+10, cont_est[mock_n], yerr=cont_err[mock_n],
                c='violet', marker='^', markersize=9,
                capsize=4, label='Stack NBs', elinewidth=2, capthick=2)
    
    plt.legend()

    plt.show()
    print(mock['EW_Arr'][mock_n])

In [ ]:
for N_nb in np.arange(1, 11):
    nb_c = 26

    pm_err = (np.ones(pm_data.shape).T * errors).T
    cont_est, cont_err = synthetic_BB_estimation(
        pm_data, pm_err, nb_c, N_nb, tcurves['t'], tcurves['w'], w_central
        )
    real_cont_value, _ = synthetic_BB_estimation(
        pm_data_noerr, pm_err, nb_c, 2, tcurves['t'], tcurves['w'], w_central, False
        )
    
    diff = np.abs((cont_est - real_cont_value) / real_cont_value)
    print('N = {} | Mean abs diff: {}'.format(N_nb, np.average(diff)))

In [ ]:
for N_nb in np.arange(1, 11):
    nb_c = 11

    pm_err = (np.ones(pm_data.shape).T * errors).T
    cont_est, cont_err = synthetic_BB_estimation(
        pm_data, pm_err, nb_c, N_nb, tcurves['t'], tcurves['w'], w_central
        )
    real_cont_value, _ = synthetic_BB_estimation(
        pm_data_noerr, pm_err, nb_c, 2, tcurves['t'], tcurves['w'], w_central, False
        )

    #### ERROR TESTING ####

    N_points = 20
    sigma_min = 0.01
    sigma_max = 4

    sigma_Arr = np.linspace(sigma_min, sigma_max, N_points)
    sigma_hist = np.zeros(N_points)

    for i in range(N_points):
        sigma_hist[i] = len(
            np.where(
                np.abs(cont_est - real_cont_value) < sigma_Arr[i] * cont_err
            )[0]
        )

    N_sources = pm_data.shape[1]

    # Plot the error test
    fig, ax =  plt.subplots(figsize = (10,10))

    ax.plot(erf(sigma_Arr / 2**0.5), sigma_hist/N_sources, marker='.', linestyle='none')
    ax.plot(erf(sigma_Arr / 2**0.5), erf(sigma_Arr / 2**0.5), linestyle='dashed')
    ax.plot(erf(sigma_Arr / 2**0.5), np.abs(sigma_hist/N_sources - erf(sigma_Arr / 2**0.5)),
    linestyle='dashed', color='dimgray')
    

    ax.set_ylabel('N / N_sources', fontsize=15)
    ax.set_xlabel('$\sigma$', fontsize=15)

    ax.set_title('N = {}'.format(N_nb), fontsize=15)

    plt.show()